# 🎬 Video Trimmer - Google Colab Edition

A simplified video trimming tool with native Google Drive integration and simple widget controls.

## Features:
- 📁 **Native Drive Integration** - Browse and select files directly from Google Drive
- 🎵 **Audio Extraction** - Automatic AAC audio extraction
- 🚀 **Fast Processing** - Uses ffmpeg for efficient video processing
- 📱 **Simple UI** - Clean widget-based interface

## Setup Instructions:
1. **Mount Google Drive** - Run the setup cell to access your files
2. **Browse Files** - Use the file picker to select your video
3. **Set Trim Points** - Use simple sliders to choose start/end times
4. **Process** - Click trim to generate output files

---

## 🔧 Setup and Mount Google Drive

In [ ]:
# Mount Google Drive and install dependencies
from google.colab import drive, files
import os
import subprocess
import ipywidgets as widgets
from IPython.display import display, HTML, Video, Audio, clear_output
import json
import tempfile
from pathlib import Path

# Mount Google Drive
print("🔗 Mounting Google Drive...")
drive.mount('/content/drive')

# Check if ffmpeg is available
try:
    result = subprocess.run(['ffmpeg', '-version'], capture_output=True, text=True)
    if result.returncode == 0:
        print("✅ ffmpeg is available and ready")
    else:
        print("❌ ffmpeg check failed")
except FileNotFoundError:
    print("❌ ffmpeg not found - installing...")
    !apt update -qq && apt install -y ffmpeg
    print("✅ ffmpeg installed")

print("\n🚀 Setup complete! You can now use the video trimmer.")

## 📁 File Browser and Selection

In [ ]:
# Simple file browser for Google Drive
import glob

def find_video_files(directory="/content/drive/MyDrive"):
    """Find video files in Google Drive"""
    video_extensions = ['*.mp4', '*.mov', '*.avi', '*.mkv', '*.MP4', '*.MOV', '*.AVI', '*.MKV']
    video_files = []
    
    for ext in video_extensions:
        pattern = os.path.join(directory, '**', ext)
        video_files.extend(glob.glob(pattern, recursive=True))
    
    return sorted(video_files)

def create_file_browser():
    """Create a simple file browser widget"""
    
    # Find video files
    video_files = find_video_files()
    
    if not video_files:
        print("❌ No video files found in Google Drive")
        print("📁 Please upload video files to your Google Drive and run this cell again")
        return None
    
    # Create dropdown for file selection
    file_options = [(os.path.basename(f), f) for f in video_files]
    
    file_selector = widgets.Dropdown(
        options=file_options,
        description='📹 Video:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='500px')
    )
    
    # Output folder selection
    output_folder = widgets.Text(
        value='/content/drive/MyDrive/Video_Trimmed',
        description='📁 Output:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='500px')
    )
    
    # File info display
    file_info = widgets.HTML(
        value="<i>Select a video file to see details</i>"
    )
    
    def update_file_info(change):
        if change['new']:
            file_path = change['new']
            file_size = os.path.getsize(file_path) / (1024 * 1024)  # MB
            file_info.value = f"<b>📊 File:</b> {os.path.basename(file_path)}<br><b>📏 Size:</b> {file_size:.1f} MB<br><b>📍 Path:</b> {file_path}"
    
    file_selector.observe(update_file_info, names='value')
    
    # Initial update
    if file_selector.value:
        update_file_info({'new': file_selector.value})
    
    print(f"📁 Found {len(video_files)} video files in Google Drive")
    display(file_selector, output_folder, file_info)
    
    return file_selector, output_folder

# Create file browser
file_browser_result = create_file_browser()
if file_browser_result:
    selected_file, output_location = file_browser_result
else:
    selected_file, output_location = None, None

## 🎥 Video Player and Duration Check

In [ ]:
def get_video_duration(video_path):
    """Get video duration using ffprobe"""
    try:
        cmd = [
            'ffprobe', '-v', 'quiet', '-print_format', 'json',
            '-show_format', '-show_streams', video_path
        ]
        result = subprocess.run(cmd, capture_output=True, text=True)
        
        if result.returncode == 0:
            data = json.loads(result.stdout)
            duration = float(data['format']['duration'])
            return duration
        return 0
    except Exception as e:
        print(f"❌ Error getting duration: {e}")
        return 0

def format_time(seconds):
    """Format seconds to MM:SS"""
    if seconds is None or seconds == 0:
        return "00:00"
    minutes = int(seconds // 60)
    secs = int(seconds % 60)
    return f"{minutes:02d}:{secs:02d}"

def create_video_player():
    """Create video player and duration display"""
    
    if not selected_file or not selected_file.value:
        print("❌ Please select a video file first")
        return None
    
    video_path = selected_file.value
    
    # Get video duration
    duration = get_video_duration(video_path)
    
    if duration == 0:
        print("❌ Could not determine video duration")
        return None
    
    # Display video player
    print(f"🎬 Video: {os.path.basename(video_path)}")
    print(f"⏱️ Duration: {format_time(duration)} ({duration:.1f} seconds)")
    
    # Show video player
    display(Video(video_path, width=600, height=400))
    
    return duration

# Create video player if file is selected
if selected_file and selected_file.value:
    video_duration = create_video_player()
else:
    print("⚠️ Please select a video file in the previous cell first")
    video_duration = None

## ✂️ Trim Controls

In [ ]:
def create_trim_controls():
    """Create simple trim control widgets"""
    
    if not video_duration:
        print("❌ Please run the video player cell first")
        return None
    
    # Time display function
    def update_time_display(start_val, end_val):
        duration_text = f"⏱️ Duration: {format_time(end_val - start_val)} ({end_val - start_val:.1f}s)"
        time_display.value = f"<b>⏯️ Start:</b> {format_time(start_val)} <b>⏹️ End:</b> {format_time(end_val)}<br>{duration_text}"
    
    # Create sliders
    start_slider = widgets.FloatSlider(
        value=0,
        min=0,
        max=video_duration,
        step=0.1,
        description='⏯️ Start:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='500px')
    )
    
    end_slider = widgets.FloatSlider(
        value=video_duration,
        min=0,
        max=video_duration,
        step=0.1,
        description='⏹️ End:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='500px')
    )
    
    # Time display
    time_display = widgets.HTML(
        value="<b>⏯️ Start:</b> 00:00 <b>⏹️ End:</b> 00:00"
    )
    
    # Update display when sliders change
    def on_slider_change(change):
        update_time_display(start_slider.value, end_slider.value)
    
    start_slider.observe(on_slider_change, names='value')
    end_slider.observe(on_slider_change, names='value')
    
    # Initial update
    update_time_display(start_slider.value, end_slider.value)
    
    # Quick preset buttons
    def create_preset_buttons():
        """Create quick preset buttons"""
        
        def set_first_30s(b):
            start_slider.value = 0
            end_slider.value = min(30, video_duration)
        
        def set_last_30s(b):
            start_slider.value = max(0, video_duration - 30)
            end_slider.value = video_duration
        
        def set_middle_30s(b):
            middle = video_duration / 2
            start_slider.value = max(0, middle - 15)
            end_slider.value = min(video_duration, middle + 15)
        
        btn_first = widgets.Button(description="📍 First 30s", button_style='info')
        btn_last = widgets.Button(description="📍 Last 30s", button_style='info')
        btn_middle = widgets.Button(description="📍 Middle 30s", button_style='info')
        
        btn_first.on_click(set_first_30s)
        btn_last.on_click(set_last_30s)
        btn_middle.on_click(set_middle_30s)
        
        return widgets.HBox([btn_first, btn_middle, btn_last])
    
    preset_buttons = create_preset_buttons()
    
    print("🎯 Drag sliders to set trim points:")
    display(start_slider, end_slider, time_display)
    
    print("\n🚀 Quick presets:")
    display(preset_buttons)
    
    return start_slider, end_slider

# Create trim controls
if video_duration:
    trim_controls = create_trim_controls()
    if trim_controls:
        start_time_slider, end_time_slider = trim_controls
else:
    print("⚠️ Please run the video player cell first")
    start_time_slider, end_time_slider = None, None

## 🚀 Video Processing

In [ ]:
def process_video():
    """Process video trimming with ffmpeg"""
    
    if not all([selected_file, start_time_slider, end_time_slider, output_location]):
        print("❌ Please complete all previous steps first")
        return
    
    input_file = selected_file.value
    start_time = start_time_slider.value
    end_time = end_time_slider.value
    output_dir = output_location.value
    
    # Validation
    if start_time >= end_time:
        print("❌ Start time must be less than end time")
        return
    
    if not os.path.exists(input_file):
        print(f"❌ Input file not found: {input_file}")
        return
    
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    # Generate output filenames
    base_name = Path(input_file).stem
    output_video = os.path.join(output_dir, f"{base_name}_trimmed.mp4")
    output_audio = os.path.join(output_dir, f"{base_name}_trimmed.aac")
    
    print(f"🎬 Processing: {os.path.basename(input_file)}")
    print(f"⏰ Trim: {format_time(start_time)} → {format_time(end_time)} ({end_time - start_time:.1f}s)")
    print(f"📁 Output directory: {output_dir}")
    
    # Convert seconds to HH:MM:SS format
    def seconds_to_hms(seconds):
        hours = int(seconds // 3600)
        minutes = int((seconds % 3600) // 60)
        secs = seconds % 60
        return f"{hours:02d}:{minutes:02d}:{secs:06.3f}"
    
    start_hms = seconds_to_hms(start_time)
    duration_hms = seconds_to_hms(end_time - start_time)
    
    # Process video
    print("\n🎥 Trimming video...")
    video_cmd = [
        'ffmpeg', '-y',
        '-ss', start_hms,
        '-i', input_file,
        '-t', duration_hms,
        '-c:v', 'libx264', '-preset', 'ultrafast', '-crf', '17',
        '-c:a', 'aac', '-b:a', '128k',
        output_video
    ]
    
    result = subprocess.run(video_cmd, capture_output=True, text=True)
    
    if result.returncode == 0:
        print("✅ Video trimmed successfully")
    else:
        print(f"❌ Video processing failed: {result.stderr}")
        return
    
    # Extract audio
    print("🎵 Extracting audio...")
    audio_cmd = [
        'ffmpeg', '-y',
        '-ss', start_hms,
        '-i', input_file,
        '-t', duration_hms,
        '-vn', '-acodec', 'aac', '-b:a', '128k',
        output_audio
    ]
    
    result = subprocess.run(audio_cmd, capture_output=True, text=True)
    
    if result.returncode == 0:
        print("✅ Audio extracted successfully")
    else:
        print(f"❌ Audio extraction failed: {result.stderr}")
        return
    
    # Display results
    print("\n🎉 Processing complete!")
    print(f"📹 Video: {output_video}")
    print(f"🎵 Audio: {output_audio}")
    
    # Show file sizes
    if os.path.exists(output_video):
        video_size = os.path.getsize(output_video) / (1024 * 1024)
        print(f"📊 Video size: {video_size:.1f} MB")
    
    if os.path.exists(output_audio):
        audio_size = os.path.getsize(output_audio) / (1024 * 1024)
        print(f"📊 Audio size: {audio_size:.1f} MB")
    
    # Display video player for result
    print("\n🎬 Result Preview:")
    display(Video(output_video, width=600, height=400))
    
    print("\n🎵 Audio Preview:")
    display(Audio(output_audio))
    
    return output_video, output_audio

# Create process button
process_button = widgets.Button(
    description="✂️ Trim Video",
    button_style='success',
    layout=widgets.Layout(width='200px', height='40px')
)

def on_process_click(b):
    with output_area:
        clear_output()
        process_video()

process_button.on_click(on_process_click)

output_area = widgets.Output()

print("🚀 Ready to process! Click the button below:")
display(process_button, output_area)

## 💾 Download Files (Optional)

If you want to download the processed files to your local machine:

In [ ]:
def download_files():
    """Download processed files to local machine"""
    
    if not output_location or not output_location.value:
        print("❌ No output location specified")
        return
    
    output_dir = output_location.value
    
    # Find processed files
    video_files = glob.glob(os.path.join(output_dir, "*_trimmed.mp4"))
    audio_files = glob.glob(os.path.join(output_dir, "*_trimmed.aac"))
    
    if not video_files and not audio_files:
        print("❌ No processed files found. Please run the trimming process first.")
        return
    
    print("📥 Downloading files...")
    
    # Download video files
    for video_file in video_files:
        if os.path.exists(video_file):
            print(f"📹 Downloading: {os.path.basename(video_file)}")
            files.download(video_file)
    
    # Download audio files
    for audio_file in audio_files:
        if os.path.exists(audio_file):
            print(f"🎵 Downloading: {os.path.basename(audio_file)}")
            files.download(audio_file)
    
    print("✅ Download complete!")

# Create download button
download_button = widgets.Button(
    description="💾 Download Files",
    button_style='info',
    layout=widgets.Layout(width='200px', height='40px')
)

download_button.on_click(lambda b: download_files())

print("💾 Download processed files to your computer:")
display(download_button)